In [ ]:
%pip install langchain langchain-openai transformers

In [ ]:

from IPython.display import Image
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# Primero autenticamos el usuario :
from google.colab import userdata
secret_string = userdata.get('OPENAI_TOKEN')
# Configurar el motor de OpenAI
engine = "gpt-4"


llm = ChatOpenAI(temperature=1, model=engine, openai_api_key=secret_string)

captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base", max_new_tokens=20)
parser = StrOutputParser()

def image2text(image):
    text_result = captioner(image)
    return text_result[0]['generated_text']

def describir_foto(topic):
   template = """
   You are a writer and story teller.
   Your task is generate short stories from a short description. The story cannot have more than 50 words.
   CONTEXT: {topic}
   STORY:
   """
   topic_template = PromptTemplate(template=template, input_variables=['topic'])
   chain = ({"topic": RunnablePassthrough()} | topic_template | llm | parser)
   #chain = LLMChain(llm=llm, prompt=topic_template)
   response = chain.invoke(topic)
   print(response)
   return response

generated_topic = image2text("https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png")
describir_foto(generated_topic)